No projeto iremos efetuar a coleta da temperatura ocorrida e prevista por dia e cidade. 
Nesta primeira etapa vamos importar as bibliotecas necessárias para construção do projeto. Será necessário tratar os seguintes pontos como normalização dos dados e utilização do webdriver para navegar entre as paginas. Também iremos carregar as variáveis, dicionários e listas. 

In [ ]:
from selenium import webdriver
import pandas
from tabulate import tabulate
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
from unidecode import unidecode
import re
from datetime import date

options = webdriver.ChromeOptions()
# options.add_argument("--headless")

driver = webdriver.Chrome(ChromeDriverManager().install(),chrome_options=options)
driver.get('https://www.accuweather.com/')

dic_mes = [  ('', 0, 0)
           , ('janeiro', 1, 31) , ('fevereiro',2, 3)   , ('março', 3, 31)
           , ('abril', 4, 30)   , ('maio', 5, 31)      , ('junho', 6, 30)
           , ('julho', 7, 31)   , ('agosto', 8, 31)    , ('setembro', 9, 30)
           , ('outubro', 10, 31), ('novembro', 11, 30) , ('dezembro', 12, 31) ]

cidade = ['Jundiaí','Itu','Guarulhos','Aracaju','Brasília','Maceió','São Paulo','Limeira'
        , 'Belo Horizonte', 'Juiz de fora', 'Teresina', 'Campinas', 'Fortaleza', 'Vinhedo' ]

colunas = ['Cidade','Data','Minima','Maximo']
# cidade = ['Jundiaí']

data_atual = date.today()


Na segunda etapa, criamos uma função que irá navegar pelo site, usaremos cidade passada na lista carregada na etapa anterior.


In [2]:
def caminho_pagina(lista_cidade):

    cola_pesquisa = []
    cola_pesquisa.append(lista_cidade)
        
    for procura_cidade in cola_pesquisa:

        pesquisa = driver.find_element_by_xpath('/html/body/div/div[1]/div[3]/div[1]/form/input')
        pesquisa.send_keys(procura_cidade)
        
        #pesquisa.send_keys(Keys.ENTER)

        time.sleep(5)

        selecionar = driver.find_element_by_xpath('/html/body/div/div[1]/div[3]/div[2]/div[2]/div[1]')
        selecionar.click()
        #driver.refresh()

        time.sleep(5)
        driver.refresh()
        time.sleep(5)

        #/html/body/div/div[1]/div[3]/div[2]/div[2]/div[1]
        mensal = driver.find_element_by_xpath('/html/body/div/div[3]/div/div[3]/a[6]')
        mensal.click()

        # driver.back()
        driver.refresh()

        mensal = driver.find_element_by_xpath('/html/body/div/div[3]/div/div[3]/a[6]')
        mensal.click()
    
        time.sleep(5)


Agora nesta terceira etapa, vamos criar a função onde irá coletar os dados utilizando com parametro o ultimo dia do mes, cidade e o dia de hoje. Todos esses parametros são carregados na primeira etapa. 

In [3]:
resultado    = []
def coleta_dados(num_ultimo_dia,lista_cidade,data_coleta):

    #vira para o mes anterior
    botao = driver.find_element_by_xpath('/html/body/div/div[5]/div[1]/div[1]/div[1]/div[1]/div[1]/div[1]')
    botao.click()

    #dic_mes[data_atual.month-1][1]
    html_mes = '/html/body/div/div[5]/div[1]/div[1]/div[1]/div[1]/div[1]/div[2]/a[' + str(data_coleta) + ']'
    mes_anetrior = driver.find_element_by_xpath(html_mes)
    mes_anetrior.click()

    num_ultimo_dia = int(num_ultimo_dia)
    lista_data   = []
    lista_maxima = []
    lista_minimo = []
    
    global resultado 
    
    for x in range(1,num_ultimo_dia):

        #identifica o primeiro dia do mes
        dia_html    = '/html/body/div/div[5]/div[1]/div[1]/div[2]/div/div[2]/a[' + str(x) + ']/div[1]/div'
        dia         = driver.find_element_by_xpath(dia_html).text

        mes = driver.find_element_by_xpath('/html/body/div/div[5]/div[1]/div[1]/div[1]/div[1]/div[1]/div[1]/h2').text
        ano = driver.find_element_by_xpath('/html/body/div/div[5]/div[1]/div[1]/div[1]/div[1]/div[2]/div[1]/h2').text

        if int(dia) == 1:
            inicio = x

            for i in range(inicio,inicio + num_ultimo_dia):

                #identifica o range a ser coletado
                dia_html    = '/html/body/div/div[5]/div[1]/div[1]/div[2]/div/div[2]/a[' + str(i) + ']/div[1]/div'
                maxima_html = '/html/body/div/div[5]/div[1]/div[1]/div[2]/div/div[2]/a[' + str(i) + ']/div[2]/div[1]'
                minima_html = '/html/body/div/div[5]/div[1]/div[1]/div[2]/div/div[2]/a[' + str(i) + ']/div[2]/div[2]'

                dia     = driver.find_element_by_xpath(dia_html).text
                maxima  = driver.find_element_by_xpath(maxima_html).text
                minima  = driver.find_element_by_xpath(minima_html).text

                lista_data.append(dia)
                lista_maxima.append(maxima)
                lista_minimo.append(minima)

    for i in range(len(lista_data)):
        tupla = (
                  unidecode(lista_cidade)
                , str(lista_data[i]) + '/' + '0' + str(data_coleta) + '/' + str(ano)
                , re.sub(r"[^a-zA-Z0-9]", "", lista_minimo[i])
                , re.sub(r"[^a-zA-Z0-9]", "", lista_maxima[i])
                )
        
        resultado.append(tupla)

Agora na quarta etapa iremos montar a execução do codigo, montando a logica para identificar qual deta será coletada e armazenar em um dataframe.

In [ ]:
# define a lista a ser utilizada, caso seja dia 1, ele busca o
# mes anterior, se nao for dia 1, ele coleta apenas o mes atual
if data_atual.day == 1:
    
    dic_data_funcao =   [
                          (dic_mes[data_atual.month-1][1], dic_mes[data_atual.month-1][2])
                        , (dic_mes[data_atual.month][1]  , dic_mes[data_atual.month][2])
                        ] 
    
else:
    dic_data_funcao =   [
                           (dic_mes[data_atual.month][1], dic_mes[data_atual.month][2])
                        ] 

for x_cidade in cidade:
    
    try:
        caminho_pagina(x_cidade)
    except Exception:
        #enviar email de notificao
        Rerotono = 'Erro na navegação!'
        break 
    
    for x_data in dic_data_funcao:
    
        try:    
            coleta_dados(x_data[1], x_cidade, x_data[0])
        except Exception:
            #enviar email de notificao
            Rerotono = 'Erro na coleta!'
        break
    inicio = driver.find_element_by_xpath('/html/body/div/div[1]/div[1]/div/a[1]')
    inicio.click()

if not resultado:
    print(Rerotono)
    driver.quit()
else:
    #precisa ser valdiado o armazenamento dos dados, pois no primeiro dia do mes, não esta coletando a data prevista
    #a oportunidade de melhoria esta no momento de armazenar o dataframe.
    tabela = pandas.DataFrame(resultado, index=None, columns=colunas)
    tabela.to_csv(r'C:\Users\lsnog\Documents\Python\ClimaTempo.csv',sep=',',index=False)
    driver.quit()